In [ ]:
import numpy as np
from scipy import linalg as la
import networkx as nx
from itertools import combinations


In [11]:
# Problems 1-2
class DiGraph:
    """A class for representing directed graphs via their adjacency matrices.

    Attributes:
        (fill this out after completing DiGraph.__init__().)
    """
    # Problem 1
    def __init__(self, A, labels=None):
        """Modify A so that there are no sinks in the corresponding graph,
        then calculate Ahat. Save Ahat and the labels as attributes.

        Parameters:
            A ((n,n) ndarray): the adjacency matrix of a directed graph.
                A[i,j] is the weight of the edge from node j to node i.
            labels (list(str)): labels for the n nodes in the graph.
                If None, defaults to [0, 1, ..., n-1].
        """
        for i in range(len(A)):
            if(np.allclose(A[:,i],np.zeros(len(A[0])))):
                #remove sink
                A[:,i] = 1
        #create ahat
        self.Ahat =  A / A.sum(axis=0)
        #get length of A
        self.n = len(A)
        #if no labels
        if labels == None:
            #Set standard
            self.labels = np.arange(0,len(A))
        else:
            #otherwise use labels
            self.labels = labels
            #throw an error if it dosn't match
        if len(A) != len(self.labels):
            raise ValueError("Number of labels is not equal to the number of nodes")


    # Problem 2
    def linsolve(self, epsilon=0.85):
        """Compute the PageRank vector using the linear system method.

        Parameters:
            epsilon (float): the damping factor, between 0 and 1.

        Returns:
            dict(str -> float): A dictionary mapping labels to PageRank values.
        """
        #find values
        p = la.solve(np.identity(self.n)-epsilon*self.Ahat,(1-epsilon)*np.ones(self.n)/self.n)
        #make a dictionary
        dict = {self.labels[i]:p[i] for i in range(self.n)}
        return dict

    # Problem 2
    def eigensolve(self, epsilon=0.85):
        """Compute the PageRank vector using the eigenvalue method.
        Normalize the resulting eigenvector so its entries sum to 1.

        Parameters:
            epsilon (float): the damping factor, between 0 and 1.

        Return:
            dict(str -> float): A dictionary mapping labels to PageRank values.
        """
        #make B
        B = epsilon*self.Ahat+(1-epsilon)/self.n*np.ones((self.n,self.n))
        #get eig stuff
        eigvals, eigvects = la.eig(B)
        #get the eig vector for the largest value
        p = eigvects[:,0].real
        #normalize it
        p = p/p.sum()
        #make a dictionary
        D = {self.labels[i]:p[i] for i in range(self.n)}
        #return the dictionary
        return D

    # Problem 2
    def itersolve(self, epsilon=0.85, maxiter=100, tol=1e-12):
        """Compute the PageRank vector using the iterative method.

        Parameters:
            epsilon (float): the damping factor, between 0 and 1.
            maxiter (int): the maximum number of iterations to compute.
            tol (float): the convergence tolerance.

        Return:
            dict(str -> float): A dictionary mapping labels to PageRank values.
        """
        t = 0
        #set p
        p = [1/self.n for n in range(self.n)]
        #while loop
        while t < maxiter:
            t += 1
            #get the next p
            pt = epsilon*self.Ahat@p + (1 - epsilon)*np.ones(self.n)/self.n
            #stop if it meets the stopping condition
            if(la.norm(pt - p, ord=1) < tol):
                break
            #otherwise reset
            p = pt
        #create the dictionary
        D = {self.labels[i]:pt[i] for i in range(self.n)}
        #return the dicitonary
        return D


# Problem 3
def get_ranks(d):
    """Construct a sorted list of labels based on the PageRank vector.

    Parameters:
        d (dict(str -> float)): a dictionary mapping labels to PageRank values.

    Returns:
        (list) the keys of d, sorted by PageRank value from greatest to least.
    """
    #get the keys
    keys = np.array(list(d.keys()))
    #get the values
    values = list(d.values())
    #create a mask
    mask = np.array(np.argsort(values)[::-1])
    #return the stuff
    return list(keys[mask])



# Problem 5
def rank_ncaa_teams(filename, epsilon=0.85):
    """Read the specified file and construct a graph where node j points to
    node i with weight w if team j was defeated by team i in w games. Use the
    DiGraph class and its itersolve() method to compute the PageRank values of
    the teams, then rank them with get_ranks().

    Each line of the file has the format
        A,B
    meaning team A defeated team B.

    Parameters:
        filename (str): the name of the data file to read.
        epsilon (float): the damping factor, between 0 and 1.

    Returns:
        (list(str)): The ranked list of team names.
    """
    labels = []
    with open(filename,'r') as infile:
        content = infile.read().strip()
    #print(content)
    labels = sorted(set(content.replace('\n',',').split(',')))
    labels.remove("Loser")
    labels.remove("Winner")
    #init dictionary
    Dict = {team: i for i, team in enumerate(labels)}
    #init adjacency matrix
    A = np.zeros((len(labels), len(labels)))
    #get the information from the lines
    for line in content.split('\n')[1:]:
        teams = line.split(',')
        row = Dict[teams[0]]
        column = Dict[teams[1]]
        A[row][column] += 1
    D = DiGraph(A, labels=labels)
    #get the page rank
    PageRank_ = D.itersolve(epsilon = epsilon)
    #return the stuff
    #print(PageRank_)
    return PageRank_, get_ranks(PageRank_)




In [8]:
!pwd

/Users/jamesgriffin/Desktop/march_madness/data


In [12]:
ranked_dict, ranks = rank_ncaa_teams('page_rank_data/ncaa2010.csv')

In [16]:
#'UConn': 0.01757875979704302,

max(list(ranked_dict.values()))

0.01757875979704302